Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Matthew Sah"
COLLABORATORS = "" 

---

# Lab 2 Introduction


In this lab you will use parse a vcf file to extracts parts of it and load it into a database. 
The fields you will be parsing from the vcf file are: 
```
CHROM
POS	
ID	
REF	
ALT	
QUAL	
FILTER
```
and from the INFO column, the following fields:
```
1000g2015aug_all
ExAC_ALL
FATHMM_pred
LRT_pred
MetaLR_pred
MetaSVM_pred
MutationAssessor_pred
MutationTaster_pred
PROVEAN_pred
Polyphen2_HDIV_pred
Polyphen2_HVAR_pred
SIFT_pred
fathmm-MKL_coding_pred.
```
The fields:
```
FATHMM_pred
LRT_pred
MetaLR_pred
MetaSVM_pred
MutationAssessor_pred
MutationTaster_pred
PROVEAN_pred
Polyphen2_HDIV_pred
Polyphen2_HVAR_pred
SIFT_pred
fathmm-MKL_coding_pred
```
are predictor fields. They use a letter to indicate whether a given variation is harmful or not.

NOTE: use the helper functions in cell2. To use them, you have to run CELL 2! 


In [1]:
## Helper functions

import os
import sqlite3
from sqlite3 import Error
import gzip

def create_connection(db_file, delete_db=False):
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)



# Part 1 (10 pts)

In part1 you will open the gzipped vcf file using the python gzip module and figure out all the possible values for the predictor fields. 

You do not have to unzip the file. Use `with gzip.open(filename,'rt') as fp:` to read one line at a time. 



In [2]:
def get_predictor_values(filename):
    """
    See part 1 description
    """
    import gzip

    keys_save_value = [
        'FATHMM_pred',
        'LRT_pred',
        'MetaLR_pred',
        'MetaSVM_pred',
        'MutationAssessor_pred',
        'MutationTaster_pred',
        'PROVEAN_pred',
        'Polyphen2_HDIV_pred',
        'Polyphen2_HVAR_pred',
        'SIFT_pred',
        'fathmm-MKL_coding_pred',
    ] 
    found_pred_line = []
    return_dict = {}
    d = {} 
    with gzip.open(filename, 'rt') as fp:
        for i in fp:  
            # Pred is in individual line
            if "pred" in i:  
                try: 
                    i_split = i.split(";")
                    
                    # Get Pred in individual position
                    for item in i_split:
                        if "pred" in item: 
                            
                            # Remove empty values
                            if not item.endswith(".") and not item.startswith("#"): 
                                found_pred_line.append(item) 
                except:
                    pass 
                
    # Retrieve from Keys
    return_dict = return_dict.fromkeys(keys_save_value, [])  
    for key in return_dict.keys(): 
        return_dict[key] = []  
    
    # Insert values into return_dict
    for values in found_pred_line :
        try:
            pred_keys_values = values.split("=")
#             print(pred_keys_values[0])
#             print(pred_keys_values[1])
            
            if pred_keys_values[0] in return_dict:
                if pred_keys_values[1] not in return_dict[pred_keys_values[0]]:
                    return_dict[pred_keys_values[0]].append(pred_keys_values[1]) 
        except:
            pass 
        
        
#     print("expected" , expected_solution)
#     print("expected1" , return_dict)
    return return_dict


# # DONT DELETE
def check_answer(expected_solution, predictor_values):
    import deepdiff
    import json

    diff = deepdiff.DeepDiff(expected_solution, predictor_values)
    print(json.dumps(diff, indent=4))

# check_answer(expected_solution, predictor_values)

In [3]:

## Can take 30 seconds to run!
expected_solution = {'SIFT_pred': ['D', 'T'], 'Polyphen2_HDIV_pred': ['D', 'B', 'P'], 'Polyphen2_HVAR_pred': ['D', 'B', 'P'], 'LRT_pred': ['D', 'N', 'U'], 'MutationTaster_pred': ['D', 'P', 'N', 'A'], 'MutationAssessor_pred': ['H', 'N', 'L', 'M'], 'FATHMM_pred': ['T', 'D'], 'PROVEAN_pred': ['D', 'N'], 'MetaSVM_pred': ['D', 'T'], 'MetaLR_pred': ['D', 'T'], 'fathmm-MKL_coding_pred': ['D', 'N']}
filename = 'test_4families_annovar.vcf.gz'
predictor_values = get_predictor_values(filename)
assert predictor_values == expected_solution



# Part 2 (No points)

You will now create 13 tables
Tables 1-11 will be for :

FATHMM_pred
LRT_pred 
MetaLR_pred
MetaSVM_pred
MutationAssessor_pred
MutationTaster_pred
PROVEAN_pred
Polyphen2_HDIV_pred
Polyphen2_HVAR_pred
SIFT_pred
fathmm_MKL_coding_pred ## NOTE: I have replaced the dash with an underscore. 

The first column for each of these tables will be the name of the field + ID, e.g., FATHMM_predID. This
column will be of type not null primary key. The second column will be called `prediction` and will be of
type TEXT not null. The prediction values will be values you extracted for each of the fields in the previous step. 
For example, 'FATHMM_pred' has two prediction values 'T' and 'D'. Name the tables after their field name, e.g.,  
call table that will contains values of FATHMM_pred `FATHMM_pred`.  Make sure to sort the values, meaning, D should 
be inserted before T. 


Table 12 will be the Variants table. 
The first column will be VariantID. 

The other columns will be:
CHROM   
POS 
ID  
REF 
ALT 
QUAL    
FILTER
thousandg2015aug_all ##NOTE: a column name cannot start with a number, so you have to rename!
ExAC_ALL


Table 12 will have 11 more columns that relate to each of prediction table. Consider writing a utility function
to fetch the primary key for a given prediction from each of the prediction table. Name each of the 
column should be the name of predictor + ID, e.g., FATHMM_predID. 


You have already deteremined their data type, so use that to set their data type. 
For integer use INTEGER. 
For float use REAL. 
For string use TEXT. 

Table 13  will be called PredictionStats. The first column will be PredictorStatsID INTEGER NOT NULL PRIMARY KEY. 
The second column will be VariantID. The third column will be PredictorName. The fourth column will be PredictorValue. 
This is not a normalized table!

The prediction value will be a float value mapped from the prediction text.  Use the following information to 
assign values: 
REF: https://brb.nci.nih.gov/seqtools/colexpanno.html#dbnsfp

FATHMM_pred
- T = 0
- D = 1

LRT_pred 
- D = 1
- N = 0
- U = 0

MetaLR_pred
- T = 0
- D = 1

MetaSVM_pred
- T = 0
- D = 1

MutationAssessor_pred
- H = 1
- N = 0
- L = 0.25
- M = 0.5

MutationTaster_pred
- D = 1
- P = 0
- N = 0
- A = 1

PROVEAN_pred
- D = 1
- N = 0

Polyphen2_HDIV_pred
- D = 1
- B = 0
- P = 0.5

Polyphen2_HVAR_pred
- D = 1
- B = 0
- P = 0.5

SIFT_pred
- D = 1
- T = 0

fathmm-MKL_coding_pred
- D = 1
- N = 0


```

prediction_mapping = {
    'FATHMM_pred': {'T': 0, 'D': 1},
    'MetaLR_pred': {'T': 0, 'D': 1},
    'MetaSVM_pred': {'T': 0, 'D': 1},
    'SIFT_pred': {'T': 0, 'D': 1},
    'fathmm_MKL_coding_pred': {'D': 1, 'N': 0},
    'LRT_pred': {'U': 0, 'N': 0, 'D': 1},
    'MutationAssessor_pred': {'H': 1, 'N': 0, 'L': 0.25, 'M': 0.5},  
    'MutationTaster_pred': {'D': 1, 'P': 0, 'N': 0, 'A': 1},
    'PROVEAN_pred': {'D': 1, 'N': 0},
    'Polyphen2_HDIV_pred': {'D': 1, 'B': 0, 'P': 0.5},
    'Polyphen2_HVAR_pred': {'D': 1, 'B': 0, 'P': 0.5},
}


```


The idea is that 11 predictors have been used to annotate all the variants in the file. This table combines all that information in one table. 
By grouping the table based on variantid and summing the prediction values mapped from the prediction text, you can find which variants have a consensus on their being deterimental. 

IMPORTANT: instead of fathmm-MKL_coding_pred use fathmm_MKL_coding_pred everywhere. 
HINT: You have to commit the changes you make to the table, otherwise your changes will not be saved. Consider wrapping all changes inside `with conn:`




In [4]:
# Creating lab2.db 
db_file = 'lab2.db'
conn = create_connection(db_file, delete_db=True)
conn.close()


In [5]:

# MY STUFF
def replace_underscore(keys_save_value):
    for q in range( 0 , len(keys_save_value)):
        keys_save_value[q] = keys_save_value[q].replace("-" , "_") 
    return keys_save_value
        
def q2_insert_into_database(db, table, column, value):
    sql = "INSERT into " + table + " ( " + column + ") VALUES (" + "'" + value+ "'" + ")"
    print(sql) 
    # # # TODO 1120
    conn = create_connection(db)
    c = conn.cursor() 
    conn.execute(sql)
    conn.commit()
    
def q2_check_value_exist(db, table, column, value):
    sql = "SELECT * FROM " + table + " WHERE " +  column +  " = " + "'" + value + "'" 

    conn = create_connection(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
#     for row in rows:
#         print(row) 
    if len(rows) == 0 :
        return False
    else: 
        return True
    
def create_tables_1_11(db_file): 
    keys_save_value = [
        'FATHMM_pred',
        'LRT_pred',
        'MetaLR_pred',
        'MetaSVM_pred',
        'MutationAssessor_pred',
        'MutationTaster_pred',
        'PROVEAN_pred',
        'Polyphen2_HDIV_pred',
        'Polyphen2_HVAR_pred',
        'SIFT_pred',
        'fathmm-MKL_coding_pred',
    ] 
    
    # replace underscore
    keys_save_value = replace_underscore(keys_save_value)
    
    
    # Connect Database
    conn = create_connection(db_file)
    c = conn.cursor()
    for i in keys_save_value: 
        create_table(conn, f"create table if not exists {i} ({i+'ID'} integer not null primary key, prediction text not null);")
    conn.commit()
    
    # Input Values 
    predictor_list =   get_predictor_values(filename)  
    for predictor_keys in predictor_list:
        tester_list = sorted(predictor_list[predictor_keys]) 
        
        # INSERT INTO DATABASE
        for values in tester_list: 
            # CHECK IF EXIST
            value_does_exist = q2_check_value_exist("lab2.db",  predictor_keys.replace("-" , "_"), "prediction", values)
            
            if not value_does_exist:
                q2_insert_into_database("lab2.db",  predictor_keys.replace("-" , "_"), "prediction", values)
#             else:
#                 print(" Does Exist")



# Excecute
db_file = 'lab2.db'    
create_tables_1_11(db_file)




INSERT into FATHMM_pred ( prediction) VALUES ('D')
INSERT into FATHMM_pred ( prediction) VALUES ('T')
INSERT into LRT_pred ( prediction) VALUES ('D')
INSERT into LRT_pred ( prediction) VALUES ('N')
INSERT into LRT_pred ( prediction) VALUES ('U')
INSERT into MetaLR_pred ( prediction) VALUES ('D')
INSERT into MetaLR_pred ( prediction) VALUES ('T')
INSERT into MetaSVM_pred ( prediction) VALUES ('D')
INSERT into MetaSVM_pred ( prediction) VALUES ('T')
INSERT into MutationAssessor_pred ( prediction) VALUES ('H')
INSERT into MutationAssessor_pred ( prediction) VALUES ('L')
INSERT into MutationAssessor_pred ( prediction) VALUES ('M')
INSERT into MutationAssessor_pred ( prediction) VALUES ('N')
INSERT into MutationTaster_pred ( prediction) VALUES ('A')
INSERT into MutationTaster_pred ( prediction) VALUES ('D')
INSERT into MutationTaster_pred ( prediction) VALUES ('N')
INSERT into MutationTaster_pred ( prediction) VALUES ('P')
INSERT into PROVEAN_pred ( prediction) VALUES ('D')
INSERT into PROV

# Part 3 (10 pts)

Write a function that returns a dictionary that maps for a given predictor its letter prediction to foreign key value. 
Conver the dash in 'fathmm-MKL_coding_pred' to an underscore. 


In [6]:
INFO_TARGET_KEYS  = [
        'FATHMM_pred',
        'LRT_pred',
        'MetaLR_pred',
        'MetaSVM_pred',
        'MutationAssessor_pred',
        'MutationTaster_pred',
        'PROVEAN_pred',
        'Polyphen2_HDIV_pred',
        'Polyphen2_HVAR_pred',
        'SIFT_pred',
        'fathmm_MKL_coding_pred',
    ] 

# RETURN DICT
def q3_return_one_to_eleven_id(db, table):
    return_dict = {}
    sql = "SELECT * FROM " + table  
    conn = create_connection(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()  
    for row in rows:
        return_dict[row[1]] = row[0]
    return return_dict
    
# MAIN FUNCTION
def get_predictor_value_to_fk_map(db_file): 
    
    return_dict = {}
    
    for key in INFO_TARGET_KEYS:
         
        # QUERY ID VALUES
        return_dict[key] = q3_return_one_to_eleven_id(db_file , key)
         
    return return_dict

# get_predictor_value_to_fk_map("lab2.db")

In [7]:
expected_solution = {
    'FATHMM_pred': {'D': 1, 'T': 2}, 
    'LRT_pred': {'D': 1, 'N': 2, 'U': 3}, 
    'MetaLR_pred': {'D': 1, 'T': 2}, 
    'MetaSVM_pred': {'D': 1, 'T': 2}, 
    'MutationAssessor_pred': {'H': 1, 'L': 2, 'M': 3, 'N': 4}, 
    'MutationTaster_pred': {'A': 1, 'D': 2, 'N': 3, 'P': 4}, 
    'PROVEAN_pred': {'D': 1, 'N': 2}, 
    'Polyphen2_HDIV_pred': {'B': 1, 'D': 2, 'P': 3}, 
    'Polyphen2_HVAR_pred': {'B': 1, 'D': 2, 'P': 3}, 
    'SIFT_pred': {'D': 1, 'T': 2}, 
    'fathmm_MKL_coding_pred': {'D': 1, 'N': 2}}

db_file = 'lab2.db'
predictor_fk_map = get_predictor_value_to_fk_map(db_file)
assert predictor_fk_map == expected_solution



# Part 4 (No Points)
Create table 12 or the variants table. See description above


In [8]:
def create_variants_table(db_file):
    conn = create_connection(db_file)
    create_table(conn,"""create table if not exists Variants 
    (VariantID INTEGER PRIMARY KEY, 
    CHROM TEXT not null, 
    POS integer, ID text, 
    REF text not null, 
    ALT text not null, 
    QUAL real  not null, 
    FILTER text  not null, 
    thousandg2015aug_all real, 
    ExAC_ALL real,
     FATHMM_predID ,
        LRT_predID,
        MetaLR_predID,
        MetaSVM_predID,
        MutationAssessor_predID,
        MutationTaster_predID,
        PROVEAN_predID,
        Polyphen2_HDIV_predID,
        Polyphen2_HVAR_predID,
        SIFT_predID,
        fathmm_MKL_coding_predID);
    """)
    conn.commit()
    
    
    # INSERT VALUE
db_file = 'lab2.db'
create_variants_table(db_file)


# Part 5 (No Points)

Create table 13 -- or the prediction stats table. See description above. 


In [9]:
def create_predictionstats_table(db_file):
    """
    Part 5
    """
    conn = create_connection(db_file)
    create_table(conn, """create table if not exists predictionStats (PredictorStatsID integer not null primary key, VariantID, PredictorName, PredictorValue);""")
    conn.commit()

db_file = 'lab2.db'
create_predictionstats_table(db_file)

# Part 6 (10 Points)

Write a function to pull the following info fields given the whole info field. 
```
values_to_pull = [
        '1000g2015aug_all',
        'ExAC_ALL',
        'FATHMM_pred',
        'LRT_pred',
        'MetaLR_pred',
        'MetaSVM_pred',
        'MutationAssessor_pred',
        'MutationTaster_pred',
        'PROVEAN_pred',
        'Polyphen2_HDIV_pred',
        'Polyphen2_HVAR_pred',
        'SIFT_pred',
        'fathmm-MKL_coding_pred',
    ]
```


In [10]:
# INIT VALUES
def values_to_pull_func():
    values_to_pull = [ 
        '1000g2015aug_all',
        'ExAC_ALL',
        'FATHMM_pred',
        'LRT_pred',
        'MetaLR_pred',
        'MetaSVM_pred',
        'MutationAssessor_pred',
        'MutationTaster_pred',
        'PROVEAN_pred',
        'Polyphen2_HDIV_pred',
        'Polyphen2_HVAR_pred',
        'SIFT_pred',
        'fathmm-MKL_coding_pred',
    ] 
    return values_to_pull


def pull_info_values(info):
    
    # INIT VAR
    values_to_pull = values_to_pull_func() 
    return_dict = {}
    
    # PARSE VALUES
    info_split = info.split(";") 
    for i in range(0 , len(info_split)):
        key_value_split = info_split[i].split("=")
        
        # FOUND TARGET PREDICTORS
        if key_value_split[0] in values_to_pull:
            
            # INSERT IN RETURN_DICT
            if key_value_split[0] == "1000g2015aug_all":
                return_dict["thousandg2015aug_all"] = key_value_split[1]
            elif key_value_split[0] == 'fathmm-MKL_coding_pred':
                return_dict['fathmm-MKL_coding_pred'.replace("-" , "_")] = key_value_split[1]
            else:
                return_dict[key_value_split[0] ] = key_value_split[1] 
                
    return return_dict
    
# sample_info_input = "AC=2;AF=0.333;AN=6;BaseQRankSum=2.23;ClippingRankSum=0;DP=131;ExcessHet=3.9794;FS=2.831;MLEAC=2;MLEAF=0.333;MQ=60;MQRankSum=0;QD=12.06;ReadPosRankSum=-0.293;SOR=0.592;VQSLOD=21.79;culprit=MQ;DB;POSITIVE_TRAIN_SITE;ANNOVAR_DATE=2018-04-16;Func.refGene=exonic;Gene.refGene=MAST2;GeneDetail.refGene=.;ExonicFunc.refGene=nonsynonymous_SNV;AAChange.refGene=MAST2:NM_015112:exon29:c.G3910A:p.V1304M;Func.ensGene=exonic;Gene.ensGene=ENSG00000086015;GeneDetail.ensGene=.;ExonicFunc.ensGene=nonsynonymous_SNV;AAChange.ensGene=ENSG00000086015:ENST00000361297:exon29:c.G3910A:p.V1304M;cytoBand=1p34.1;gwasCatalog=.;tfbsConsSites=.;wgRna=.;targetScanS=.;Gene_symbol=.;OXPHOS_Complex=.;Ensembl_Gene_ID=.;Ensembl_Protein_ID=.;Uniprot_Name=.;Uniprot_ID=.;NCBI_Gene_ID=.;NCBI_Protein_ID=.;Gene_pos=.;AA_pos=.;AA_sub=.;Codon_sub=.;dbSNP_ID=.;PhyloP_46V=.;PhastCons_46V=.;PhyloP_100V=.;PhastCons_100V=.;SiteVar=.;PolyPhen2_prediction=.;PolyPhen2_score=.;SIFT_prediction=.;SIFT_score=.;FatHmm_prediction=.;FatHmm_score=.;PROVEAN_prediction=.;PROVEAN_score=.;MutAss_prediction=.;MutAss_score=.;EFIN_Swiss_Prot_Score=.;EFIN_Swiss_Prot_Prediction=.;EFIN_HumDiv_Score=.;EFIN_HumDiv_Prediction=.;CADD_score=.;CADD_Phred_score=.;CADD_prediction=.;Carol_prediction=.;Carol_score=.;Condel_score=.;Condel_pred=.;COVEC_WMV=.;COVEC_WMV_prediction=.;PolyPhen2_score_transf=.;PolyPhen2_pred_transf=.;SIFT_score_transf=.;SIFT_pred_transf=.;MutAss_score_transf=.;MutAss_pred_transf=.;Perc_coevo_Sites=.;Mean_MI_score=.;COSMIC_ID=.;Tumor_site=.;Examined_samples=.;Mutation_frequency=.;US=.;Status=.;Associated_disease=.;Presence_in_TD=.;Class_predicted=.;Prob_N=.;Prob_P=.;SIFT_score=0.034;SIFT_converted_rankscore=0.440;SIFT_pred=D;Polyphen2_HDIV_score=0.951;Polyphen2_HDIV_rankscore=0.520;Polyphen2_HDIV_pred=P;Polyphen2_HVAR_score=0.514;Polyphen2_HVAR_rankscore=0.462;Polyphen2_HVAR_pred=P;LRT_score=0.002;LRT_converted_rankscore=0.368;LRT_pred=N;MutationTaster_score=1.000;MutationTaster_converted_rankscore=0.810;MutationTaster_pred=D;MutationAssessor_score=1.67;MutationAssessor_score_rankscore=0.430;MutationAssessor_pred=L;FATHMM_score=1.36;FATHMM_converted_rankscore=0.344;FATHMM_pred=T;PROVEAN_score=-1.4;PROVEAN_converted_rankscore=0.346;PROVEAN_pred=N;VEST3_score=0.158;VEST3_rankscore=0.189;MetaSVM_score=-1.142;MetaSVM_rankscore=0.013;MetaSVM_pred=T;MetaLR_score=0.008;MetaLR_rankscore=0.029;MetaLR_pred=T;M-CAP_score=.;M-CAP_rankscore=.;M-CAP_pred=.;CADD_raw=4.716;CADD_raw_rankscore=0.632;CADD_phred=24.6;DANN_score=0.998;DANN_rankscore=0.927;fathmm-MKL_coding_score=0.900;fathmm-MKL_coding_rankscore=0.506;fathmm-MKL_coding_pred=D;Eigen_coding_or_noncoding=c;Eigen-raw=0.461;Eigen-PC-raw=0.469;GenoCanyon_score=1.000;GenoCanyon_score_rankscore=0.747;integrated_fitCons_score=0.672;integrated_fitCons_score_rankscore=0.522;integrated_confidence_value=0;GERP++_RS=4.22;GERP++_RS_rankscore=0.490;phyloP100way_vertebrate=4.989;phyloP100way_vertebrate_rankscore=0.634;phyloP20way_mammalian=1.047;phyloP20way_mammalian_rankscore=0.674;phastCons100way_vertebrate=1.000;phastCons100way_vertebrate_rankscore=0.715;phastCons20way_mammalian=0.999;phastCons20way_mammalian_rankscore=0.750;SiPhy_29way_logOdds=17.151;SiPhy_29way_logOdds_rankscore=0.866;Interpro_domain=.;GTEx_V6_gene=ENSG00000162415.6;GTEx_V6_tissue=Nerve_Tibial;esp6500siv2_all=0.0560;esp6500siv2_aa=0.0160;esp6500siv2_ea=0.0761;ExAC_ALL=0.0553;ExAC_AFR=0.0140;ExAC_AMR=0.0386;ExAC_EAS=0.0005;ExAC_FIN=0.0798;ExAC_NFE=0.0788;ExAC_OTH=0.0669;ExAC_SAS=0.0145;ExAC_nontcga_ALL=0.0541;ExAC_nontcga_AFR=0.0129;ExAC_nontcga_AMR=0.0379;ExAC_nontcga_EAS=0.0004;ExAC_nontcga_FIN=0.0798;ExAC_nontcga_NFE=0.0802;ExAC_nontcga_OTH=0.0716;ExAC_nontcga_SAS=0.0144;ExAC_nonpsych_ALL=0.0496;ExAC_nonpsych_AFR=0.0140;ExAC_nonpsych_AMR=0.0386;ExAC_nonpsych_EAS=0.0005;ExAC_nonpsych_FIN=0.0763;ExAC_nonpsych_NFE=0.0785;ExAC_nonpsych_OTH=0.0638;ExAC_nonpsych_SAS=0.0145;1000g2015aug_all=0.024361;1000g2015aug_afr=0.0038;1000g2015aug_amr=0.0461;1000g2015aug_eur=0.0795;1000g2015aug_sas=0.0041;CLNALLELEID=.;CLNDN=.;CLNDISDB=.;CLNREVSTAT=.;CLNSIG=.;dbscSNV_ADA_SCORE=.;dbscSNV_RF_SCORE=.;snp138NonFlagged=rs33931638;avsnp150=rs33931638;CADD13_RawScore=4.716301;CADD13_PHRED=24.6;Eigen=0.4614;REVEL=0.098;MCAP=.;Interpro_domain=.;ICGC_Id=.;ICGC_Occurrence=.;gnomAD_genome_ALL=0.0507;gnomAD_genome_AFR=0.0114;gnomAD_genome_AMR=0.0430;gnomAD_genome_ASJ=0.1159;gnomAD_genome_EAS=0;gnomAD_genome_FIN=0.0802;gnomAD_genome_NFE=0.0702;gnomAD_genome_OTH=0.0695;gerp++gt2=4.22;cosmic70=.;InterVar_automated=Benign;PVS1=0;PS1=0;PS2=0;PS3=0;PS4=0;PM1=0;PM2=0;PM3=0;PM4=0;PM5=0;PM6=0;PP1=0;PP2=0;PP3=0;PP4=0;PP5=0;BA1=1;BS1=1;BS2=0;BS3=0;BS4=0;BP1=0;BP2=0;BP3=0;BP4=0;BP5=0;BP6=0;BP7=0;Kaviar_AF=0.0552127;Kaviar_AC=8536;Kaviar_AN=154602;ALLELE_END"
# my_solution = pull_info_values(sample_info_input)
 

In [11]:
sample_info_input = "AC=2;AF=0.333;AN=6;BaseQRankSum=2.23;ClippingRankSum=0;DP=131;ExcessHet=3.9794;FS=2.831;MLEAC=2;MLEAF=0.333;MQ=60;MQRankSum=0;QD=12.06;ReadPosRankSum=-0.293;SOR=0.592;VQSLOD=21.79;culprit=MQ;DB;POSITIVE_TRAIN_SITE;ANNOVAR_DATE=2018-04-16;Func.refGene=exonic;Gene.refGene=MAST2;GeneDetail.refGene=.;ExonicFunc.refGene=nonsynonymous_SNV;AAChange.refGene=MAST2:NM_015112:exon29:c.G3910A:p.V1304M;Func.ensGene=exonic;Gene.ensGene=ENSG00000086015;GeneDetail.ensGene=.;ExonicFunc.ensGene=nonsynonymous_SNV;AAChange.ensGene=ENSG00000086015:ENST00000361297:exon29:c.G3910A:p.V1304M;cytoBand=1p34.1;gwasCatalog=.;tfbsConsSites=.;wgRna=.;targetScanS=.;Gene_symbol=.;OXPHOS_Complex=.;Ensembl_Gene_ID=.;Ensembl_Protein_ID=.;Uniprot_Name=.;Uniprot_ID=.;NCBI_Gene_ID=.;NCBI_Protein_ID=.;Gene_pos=.;AA_pos=.;AA_sub=.;Codon_sub=.;dbSNP_ID=.;PhyloP_46V=.;PhastCons_46V=.;PhyloP_100V=.;PhastCons_100V=.;SiteVar=.;PolyPhen2_prediction=.;PolyPhen2_score=.;SIFT_prediction=.;SIFT_score=.;FatHmm_prediction=.;FatHmm_score=.;PROVEAN_prediction=.;PROVEAN_score=.;MutAss_prediction=.;MutAss_score=.;EFIN_Swiss_Prot_Score=.;EFIN_Swiss_Prot_Prediction=.;EFIN_HumDiv_Score=.;EFIN_HumDiv_Prediction=.;CADD_score=.;CADD_Phred_score=.;CADD_prediction=.;Carol_prediction=.;Carol_score=.;Condel_score=.;Condel_pred=.;COVEC_WMV=.;COVEC_WMV_prediction=.;PolyPhen2_score_transf=.;PolyPhen2_pred_transf=.;SIFT_score_transf=.;SIFT_pred_transf=.;MutAss_score_transf=.;MutAss_pred_transf=.;Perc_coevo_Sites=.;Mean_MI_score=.;COSMIC_ID=.;Tumor_site=.;Examined_samples=.;Mutation_frequency=.;US=.;Status=.;Associated_disease=.;Presence_in_TD=.;Class_predicted=.;Prob_N=.;Prob_P=.;SIFT_score=0.034;SIFT_converted_rankscore=0.440;SIFT_pred=D;Polyphen2_HDIV_score=0.951;Polyphen2_HDIV_rankscore=0.520;Polyphen2_HDIV_pred=P;Polyphen2_HVAR_score=0.514;Polyphen2_HVAR_rankscore=0.462;Polyphen2_HVAR_pred=P;LRT_score=0.002;LRT_converted_rankscore=0.368;LRT_pred=N;MutationTaster_score=1.000;MutationTaster_converted_rankscore=0.810;MutationTaster_pred=D;MutationAssessor_score=1.67;MutationAssessor_score_rankscore=0.430;MutationAssessor_pred=L;FATHMM_score=1.36;FATHMM_converted_rankscore=0.344;FATHMM_pred=T;PROVEAN_score=-1.4;PROVEAN_converted_rankscore=0.346;PROVEAN_pred=N;VEST3_score=0.158;VEST3_rankscore=0.189;MetaSVM_score=-1.142;MetaSVM_rankscore=0.013;MetaSVM_pred=T;MetaLR_score=0.008;MetaLR_rankscore=0.029;MetaLR_pred=T;M-CAP_score=.;M-CAP_rankscore=.;M-CAP_pred=.;CADD_raw=4.716;CADD_raw_rankscore=0.632;CADD_phred=24.6;DANN_score=0.998;DANN_rankscore=0.927;fathmm-MKL_coding_score=0.900;fathmm-MKL_coding_rankscore=0.506;fathmm-MKL_coding_pred=D;Eigen_coding_or_noncoding=c;Eigen-raw=0.461;Eigen-PC-raw=0.469;GenoCanyon_score=1.000;GenoCanyon_score_rankscore=0.747;integrated_fitCons_score=0.672;integrated_fitCons_score_rankscore=0.522;integrated_confidence_value=0;GERP++_RS=4.22;GERP++_RS_rankscore=0.490;phyloP100way_vertebrate=4.989;phyloP100way_vertebrate_rankscore=0.634;phyloP20way_mammalian=1.047;phyloP20way_mammalian_rankscore=0.674;phastCons100way_vertebrate=1.000;phastCons100way_vertebrate_rankscore=0.715;phastCons20way_mammalian=0.999;phastCons20way_mammalian_rankscore=0.750;SiPhy_29way_logOdds=17.151;SiPhy_29way_logOdds_rankscore=0.866;Interpro_domain=.;GTEx_V6_gene=ENSG00000162415.6;GTEx_V6_tissue=Nerve_Tibial;esp6500siv2_all=0.0560;esp6500siv2_aa=0.0160;esp6500siv2_ea=0.0761;ExAC_ALL=0.0553;ExAC_AFR=0.0140;ExAC_AMR=0.0386;ExAC_EAS=0.0005;ExAC_FIN=0.0798;ExAC_NFE=0.0788;ExAC_OTH=0.0669;ExAC_SAS=0.0145;ExAC_nontcga_ALL=0.0541;ExAC_nontcga_AFR=0.0129;ExAC_nontcga_AMR=0.0379;ExAC_nontcga_EAS=0.0004;ExAC_nontcga_FIN=0.0798;ExAC_nontcga_NFE=0.0802;ExAC_nontcga_OTH=0.0716;ExAC_nontcga_SAS=0.0144;ExAC_nonpsych_ALL=0.0496;ExAC_nonpsych_AFR=0.0140;ExAC_nonpsych_AMR=0.0386;ExAC_nonpsych_EAS=0.0005;ExAC_nonpsych_FIN=0.0763;ExAC_nonpsych_NFE=0.0785;ExAC_nonpsych_OTH=0.0638;ExAC_nonpsych_SAS=0.0145;1000g2015aug_all=0.024361;1000g2015aug_afr=0.0038;1000g2015aug_amr=0.0461;1000g2015aug_eur=0.0795;1000g2015aug_sas=0.0041;CLNALLELEID=.;CLNDN=.;CLNDISDB=.;CLNREVSTAT=.;CLNSIG=.;dbscSNV_ADA_SCORE=.;dbscSNV_RF_SCORE=.;snp138NonFlagged=rs33931638;avsnp150=rs33931638;CADD13_RawScore=4.716301;CADD13_PHRED=24.6;Eigen=0.4614;REVEL=0.098;MCAP=.;Interpro_domain=.;ICGC_Id=.;ICGC_Occurrence=.;gnomAD_genome_ALL=0.0507;gnomAD_genome_AFR=0.0114;gnomAD_genome_AMR=0.0430;gnomAD_genome_ASJ=0.1159;gnomAD_genome_EAS=0;gnomAD_genome_FIN=0.0802;gnomAD_genome_NFE=0.0702;gnomAD_genome_OTH=0.0695;gerp++gt2=4.22;cosmic70=.;InterVar_automated=Benign;PVS1=0;PS1=0;PS2=0;PS3=0;PS4=0;PM1=0;PM2=0;PM3=0;PM4=0;PM5=0;PM6=0;PP1=0;PP2=0;PP3=0;PP4=0;PP5=0;BA1=1;BS1=1;BS2=0;BS3=0;BS4=0;BP1=0;BP2=0;BP3=0;BP4=0;BP5=0;BP6=0;BP7=0;Kaviar_AF=0.0552127;Kaviar_AC=8536;Kaviar_AN=154602;ALLELE_END"

expected_solution = {
    'thousandg2015aug_all': '0.024361', 
    'ExAC_ALL': '0.0553', 
    'SIFT_pred': 'D', 
    'Polyphen2_HDIV_pred': 'P', 
    'Polyphen2_HVAR_pred': 'P', 
    'LRT_pred': 'N',
    'MutationTaster_pred': 'D', 
    'MutationAssessor_pred': 'L', 
    'FATHMM_pred': 'T', 
    'PROVEAN_pred': 'N', 
    'MetaSVM_pred': 'T', 
    'MetaLR_pred': 'T', 
    'fathmm_MKL_coding_pred': 'D'
}


solution  = pull_info_values(sample_info_input)
assert solution == expected_solution


# Part 7 (10 points)

Remember that to insert a record in SQLite, you have to use `cur.execute(sql, values)`, where `sql` is the insert statement and `values` is a list/tuple of values that will be substituted into the `sql` string wherever there is a question mark. 

Write a function that takes in as input: CHROM, POS, ID, REF, ALT, QUAL, FILTER, info_values and returns a list with the values in the following order:
```
CHROM 
POS
ID
REF
ALT
QUAL
FILTER
thousandg2015aug_all # 1000 has been replaced by the text thousand
ExAC_ALL
FATHMM_pred
LRT_pred
MetaLR_pred
MetaSVM_pred
MutationAssessor_pred
MutationTaster_pred
PROVEAN_pred
Polyphen2_HDIV_pred
Polyphen2_HVAR_pred
SIFT_pred
fathmm_MKL_coding_pred # note that the dash has been replaced by underscore
```

The info_values dictionary contains the predictor values. Use `None` for any empty/missing value for info fields, thousandg2015aug_all, and ExAC_ALL. 



In [12]:

def build_values_list(CHROM, POS, ID, REF, ALT, QUAL, FILTER, info_values):
     
    # ADD INFO_VALUES HERE
    return_list = p7_return_base_list(CHROM, POS, ID, REF, ALT, QUAL, FILTER )
    
    # INIT INFO_VALUE LIST ORDER
    info_list_order = p7_init_list_order()
    
    db_file = "lab2.db"
    
    # LOOP THROUGH ORDER LIST TO PARSE INFO_VALUES
    for keys_in_order in info_list_order:
        
        return_value = "" 
        
        try:
            # ALPHABETICAL VALUE 
            to_query_value = info_values[keys_in_order]
            
            # COLUMN OF VALUE
            to_query_column = keys_in_order + "ID" 
            
            to_query_table = keys_in_order
            
            # QUERY NUMERICAL VALUE FROM DB 
            return_value = p7_query_for_numerical_value(db_file , to_query_table, to_query_column , to_query_value )
            
            # ADD NUMERICAL VALUE TO LIST
            return_list.append(return_value)
            
            
        except Exception as e:  
            try: 
                if info_values[keys_in_order] == ".":
                    return_list.append(None)
                else:
                    return_list.append(info_values[keys_in_order])
            except:
                return_list.append(None)  
    return return_list
    
# QUERY DB FOR PREDICTOR ID
def p7_query_for_numerical_value(db_file, table, target_column, alphabetical_value):
    
    # Connect Database
    conn = create_connection(db_file)
    c = conn.cursor() 
    sql = "SELECT " + target_column + " FROM " + table + " WHERE " + "prediction " + " = " + "'" + alphabetical_value + "' "
    c.execute( sql)
    conn.commit()
    rows = c.fetchall() 
    queried_prediction_id = rows[0][0]
    return queried_prediction_id
    
    
    
# RETURN LIST BASE
def p7_return_base_list(CHROM, POS, ID, REF, ALT, QUAL, FILTER ):
    return_list = []
    return_list.append(CHROM )
    return_list.append(POS)
    return_list.append(ID )
    return_list.append(REF)
    return_list.append(ALT )
    return_list.append(QUAL)
    return_list.append(FILTER)
    return return_list
    

def p7_init_list_order():
    list_order = ["thousandg2015aug_all", # 1000 has been replaced by the text thousand
        "ExAC_ALL",
        "FATHMM_pred",
        "LRT_pred",
        "MetaLR_pred",
        "MetaSVM_pred",
        "MutationAssessor_pred",
        "MutationTaster_pred",
        "PROVEAN_pred",
        "Polyphen2_HDIV_pred",
        "Polyphen2_HVAR_pred",
        "SIFT_pred",
        "fathmm_MKL_coding_pred" 
    ]
    return list_order

    
#  # === TEST CODE ===

# p7_init_list_order()

# CHROM, POS, ID, REF, ALT, QUAL, FILTER = (7, 87837848, '.', 'C', 'A', 418.25, 'PASS') 

# # OUTPUT FROM Q6
# info_values = {'SIFT_pred': 'D', "thousandg2015aug_all" : 0.0251597, "ExAC_ALL" :0.0425,'Polyphen2_HDIV_pred': 'D', 'Polyphen2_HVAR_pred': 'D', 'LRT_pred': 'D', 'MutationTaster_pred': 'D', 'MutationAssessor_pred': 'H', 'FATHMM_pred': 'T', 'PROVEAN_pred': 'D', 'MetaSVM_pred': 'D', 'MetaLR_pred': 'D', 'fathmm_MKL_coding_pred': 'D'}

# results = build_values_list(CHROM, POS, ID, REF, ALT, QUAL, FILTER, info_values)
# expected_results = [7, 87837848, '.', 'C', 'A', 418.25, 'PASS', 0.0251597 ,0.0425 ,  2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1]
# assert results == expected_results



In [13]:
CHROM, POS, ID, REF, ALT, QUAL, FILTER = (7, 87837848, '.', 'C', 'A', 418.25, 'PASS') 
info_values = {'SIFT_pred': 'D', 'Polyphen2_HDIV_pred': 'D', 'Polyphen2_HVAR_pred': 'D', 'LRT_pred': 'D', 'MutationTaster_pred': 'D', 'MutationAssessor_pred': 'H', 'FATHMM_pred': 'T', 'PROVEAN_pred': 'D', 'MetaSVM_pred': 'D', 'MetaLR_pred': 'D', 'fathmm_MKL_coding_pred': 'D'}

results = build_values_list(CHROM, POS, ID, REF, ALT, QUAL, FILTER, info_values)
expected_results = [7, 87837848, '.', 'C', 'A', 418.25, 'PASS', None, None, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1]
assert results == expected_results


# Part 8 (No Points)

Create a function that takes the database `conn` and `values` from the `build_values_list` function to insert a variant record. 

IMPORTANT: Function should return the id of the row inserted, which will be VariantId


In [14]:
def q8_determine_data_type(value):
    
    try :
        temp_value = float(value) 
        try:
            temp_value = int(value) 
            return int
        except:  
            return float
    except:
        return str
    
    raise NotImplementedError()
    
def q8_init_insert_sql(): 
    insert_sql = '''INSERT INTO Variants(CHROM, POS, ID, REF, ALT, QUAL, FILTER, thousandg2015aug_all, ExAC_ALL
    , FATHMM_predID,
        LRT_predID,
        MetaLR_predID,
        MetaSVM_predID,
        MutationAssessor_predID,
        MutationTaster_predID,
        PROVEAN_predID,
        Polyphen2_HDIV_predID,
        Polyphen2_HVAR_predID,
        SIFT_predID,
        fathmm_MKL_coding_predID) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''
    
#     print("insert_sql " , insert_sql)
    return insert_sql
    
def q8_init_select_sql():
    select_sql =  ''' SELECT VariantID FROM Variants WHERE  
    CHROM  ? 
    and POS   ? 
    and ID  ? 
    and REF  ? 
    and ALT   ? 
    and QUAL  ?
    and FILTER  ? 
    and thousandg2015aug_all ?
    and ExAC_ALL ?
    and FATHMM_predID  ? 
    and LRT_predID  ? 
    and MetaLR_predID  ?
    and MetaSVM_predID  ? 
    and MutationAssessor_predID  ?
    and MutationTaster_predID  ?
    and PROVEAN_predID  ? 
    and Polyphen2_HDIV_predID  ? 
    and Polyphen2_HVAR_predID  ?
    and SIFT_predID  ?
    and fathmm_MKL_coding_predID  ? ''' 
    return select_sql
    
    
def insert_variant(conn, values): 
            
    # INSERT INTO SQL
    insert_sql = q8_init_insert_sql()
    c = conn.cursor()
    c.execute(insert_sql,values)
    conn.commit()
    
     
    new_value = []
    for value in values:
        if value != None: 
            value_type = q8_determine_data_type(value)
            append_string = ""
            if value_type == int or value_type == float:
                append_string = " = " + str(value)
            elif value_type == str:
                append_string = " = '" + str(value) + "'"
            new_value.append(append_string)
        else :
            new_value.append(" is null ")  
    
#     print("values" , values)
#     print("new values " ,new_value)
            
    # RETRIEVE INSERTED VALUE VARIANT_ID
    select_sql  = q8_init_select_sql()
    for i in range(0, len(new_value)): 
        select_sql = select_sql.replace("?" , new_value[i], 1) 
#     print(select_sql)
    c.execute(select_sql)
    conn.commit()
    rows = c.fetchall()  
    return_dict = {} 
    return rows[0][0] 
     
    
    
# # TEST CODE


# db_file = 'lab2.db'
# conn = create_connection(db_file)
# output_from_7 = [7, 87837848, '.', 'C', 'A', 418.25, 'PASS', None, None, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1]
# insert_variant(conn, output_from_7)

# Part 9 (No Points)

Create a function that takes the database `conn` and `values` which is the tuple `(VariantId, PredictorName, PredictorValue)` Where `VariantId` will be the return value from the `insert_variant` function,  `PredictorName` is the name of the predictor, and `PredictorValue` is the mapping of the text value to a numeric value. 


In [15]:
# def insert_predictionstat(conn, values):
#     """
#     See description in part 9
#     """
#     # YOUR CODE HERE
#     raise NotImplementedError()


def insert_predictionstat(conn, values):
    """
    See description in part 9
    """
    sql = '''insert into predictionStats(VariantID, PredictorName, PredictorValue) VALUES(?,?,?)'''
 
    c = conn.cursor()
    c.execute(sql,values)
    conn.commit()
    
# # TEST CODE

# db_file = 'lab2.db'
# conn = create_connection(db_file)
# tuple_predictor_stats = (49843, "SIFT_pred", 0.0)
# insert_predictionstat(conn,tuple_predictor_stats )

# Part 10 (No Points)

Write a function to insert records into both the variants and predictor_stats table. 
Hint:
1) Open connection to database  
2) Read file one line at a time using gzip read
3) Extract CHROM, POS, ID, REF, ALT, QUAL, FILTER, INFO
4) Use the pull_info_values function
5) Use build_values_list 
6) Use insert_variant -- save variant_id
7) Use insert_predictionstat -- insert each predictor at a time and remember to use `prediction_mapping` mapping. 




```

prediction_mapping = {
    'FATHMM_pred': {'T': 0, 'D': 1},
    'MetaLR_pred': {'T': 0, 'D': 1},
    'MetaSVM_pred': {'T': 0, 'D': 1},
    'SIFT_pred': {'T': 0, 'D': 1},
    'fathmm_MKL_coding_pred': {'D': 1, 'N': 0},
    'LRT_pred': {'U': 0, 'N': 0, 'D': 1},
    'MutationAssessor_pred': {'H': 1, 'N': 0, 'L': 0.25, 'M': 0.5},  
    'MutationTaster_pred': {'D': 1, 'P': 0, 'N': 0, 'A': 1},
    'PROVEAN_pred': {'D': 1, 'N': 0},
    'Polyphen2_HDIV_pred': {'D': 1, 'B': 0, 'P': 0.5},
    'Polyphen2_HVAR_pred': {'D': 1, 'B': 0, 'P': 0.5},
}


```


In [20]:
prediction_mapping = {
    'FATHMM_pred': {'T': 0, 'D': 1},
    'MetaLR_pred': {'T': 0, 'D': 1},
    'MetaSVM_pred': {'T': 0, 'D': 1},
    'SIFT_pred': {'T': 0, 'D': 1},
    'fathmm_MKL_coding_pred': {'D': 1, 'N': 0},
    'LRT_pred': {'U': 0, 'N': 0, 'D': 1},
    'MutationAssessor_pred': {'H': 1, 'N': 0, 'L': 0.25, 'M': 0.5},  
    'MutationTaster_pred': {'D': 1, 'P': 0, 'N': 0, 'A': 1},
    'PROVEAN_pred': {'D': 1, 'N': 0},
    'Polyphen2_HDIV_pred': {'D': 1, 'B': 0, 'P': 0.5},
    'Polyphen2_HVAR_pred': {'D': 1, 'B': 0, 'P': 0.5},
} 

def populate_variants_predictorstats_tables(db_file, filename):
     # # # 1) Open connection to database
     
    conn = create_connection(db_file)
    c = conn.cursor()    
    
    # # # 2) Read file one line at a time using gzip  
    extract_values =  ["CHROM" , "POS" , "ID" , "REF" , "ALT" , "QUAL" , "FILTER" , "INFO"]
    
    # TESTER CODE
    index_count = 0
    
    with gzip.open(filename, 'rt') as fp: 
        for lines in fp:  
#             print("index, " , index) 
            
            # DATA STARTS HERE
            if "##" not in lines and lines[0] != "#": #and index_count <10   
#                 times += 1
                extracted_values_dict = {}
                
                # # # 3) Extract CHROM, POS, ID, REF, ALT, QUAL, FILTER, INFO 
                split_lines = lines.split("\t")
                
                for extractor in range(0 , len(extract_values)):
                    extracted_values_dict[extract_values[extractor]] = split_lines[extractor]
#                 print("extracted_values_dict" , extracted_values_dict)
                
                # # # 4) Use the pull_info_values function 
                input_pull_info_values = extracted_values_dict["INFO"]
                
                
                pulled_info_values  = pull_info_values(input_pull_info_values)
                # pulled_info_values = {
                #     'thousandg2015aug_all': '0.024361', 
                #     'ExAC_ALL': '0.0553', 
                #     'SIFT_pred': 'D', 
                #     'Polyphen2_HDIV_pred': 'P', 
                #     'Polyphen2_HVAR_pred': 'P', 
                #     'LRT_pred': 'N',
                #     'MutationTaster_pred': 'D', 
                #     'MutationAssessor_pred': 'L', 
                #     'FATHMM_pred': 'T', 
                #     'PROVEAN_pred': 'N', 
                #     'MetaSVM_pred': 'T', 
                #     'MetaLR_pred': 'T', 
                #     'fathmm_MKL_coding_pred': 'D'
                # }
#                 print(pulled_info_values)
                
    
                # # # 5) Use build_values_list  
                built_value_list = build_values_list(extracted_values_dict["CHROM"], extracted_values_dict["POS"], extracted_values_dict["ID"], extracted_values_dict["REF"], extracted_values_dict["ALT"], extracted_values_dict["QUAL"], extracted_values_dict["FILTER"], pulled_info_values)
                # built_value_list = [7, 87837848, '.', 'C', 'A', 418.25, 'PASS', None, None, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1]
#                 print(built_value_list)
                
#                 print(temp_tuple)
                # # # 6) Use insert_variant -- save variant_id  
                
#                 if pulled_info_values["ExAC_ALL"] is not ".": 
#                     print("!! " , pulled_info_values["ExAC_ALL"]) 
#                     print("Value List " , built_value_list)
        
                variant_id = insert_variant(conn, built_value_list)
                
                # # # 7) Use insert_predictionstat -- insert each predictor at a time and remember to use prediction_mapping mapping. 
                # # USE DATA FROM STEP 4
                for predictor in prediction_mapping:
                    try: 
                        # CHECK VALUE FROM pulled_info_values WITH prediction_mapping
                        to_check_info_value = pulled_info_values[predictor]
#                         print("to_check_info_value" , to_check_info_value)

                        found_numerical_value = prediction_mapping[predictor][to_check_info_value]
#                         print("found_numerical_value", found_numerical_value)
#                         print("to_check_info_value" , to_check_info_value)

                        tuple_predictor_stats = (variant_id , predictor , found_numerical_value)
#                         print("tuple_predictor_stats" , tuple_predictor_stats)

                        insert_predictionstat(conn,tuple_predictor_stats )
#                         print("success?")
                    except Exception as e:
#                         print("Exception" ,e)
                        pass 

    
    
try:
    conn.close()
except:
    pass

db_file = 'lab2.db'
filename = 'test_4families_annovar.vcf.gz'

import time
time_start = time.time()
populate_variants_predictorstats_tables(db_file, filename)
end_time = time.time()
total = end_time - time_start
print("elapsed time " , total/60)

elapsed time  6.861192965507508


# Part 11 (10 Points)

Write a function that returns the total number of variants



In [21]:
def num_of_total_variants(conn):
    
    sum_sql = q11_find_sum_sql()
    c = conn.cursor()   
    c.execute(sum_sql)
    conn.commit()
    rows = c.fetchall()  
    print(rows[0][0])
    return rows[0][0]
    
def q11_find_sum_sql():
    sum_sql = "SELECT COUNT(VariantID) FROM Variants"
    
    return sum_sql
    


In [22]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert num_of_total_variants(conn) == 50001
conn.close()

50001


# Part 12 (10 Points)
Write a function returns the total number of variant predictions -- the count of the predictiostats table. 

In [23]:
def num_of_total_variant_predictions(conn):
    predictor_sum_sql = q12_total_variant()
    c = conn.cursor()
    c.execute(predictor_sum_sql)
    conn.commit()
    rows = c.fetchall() 
    return rows[0][0]
    
def q12_total_variant():
    predictor_sum_sql = "SELECT COUNT(PredictorStatsID) FROM predictionStats"
    return predictor_sum_sql


In [24]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert num_of_total_variant_predictions(conn) == 1324
conn.close()

# Part 13 (10 Points)
Return the total number of variant predictions that have value greater than zero. Number of values from the predictionstats table that are greater than 0. 

In [25]:
def num_of_total_variant_predictions_with_value_gt_zero(conn): 
    query_sql = q13_gt_zero_count_sql()
    c = conn.cursor()
    c.execute(query_sql)
    conn.commit()
    rows = c.fetchall() 
    return rows[0][0]
    
def q13_gt_zero_count_sql():
    gt_zero_predictor_sql = " SELECT COUNT(PredictorValue) FROM predictionStats WHERE PredictorValue > 0"
    return gt_zero_predictor_sql

In [26]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert num_of_total_variant_predictions_with_value_gt_zero(conn) == 219
conn.close()

# Part 14 (10 Points)

Write a function that given `CHROM, POS, ID, REF, ALT` returns a variant's info with the following columns (column order is important) :
```
Variants.CHROM,
Variants.POS,
Variants.ID,
Variants.REF,
Variants.ALT,
Variants.QUAL,
Variants.FILTER,
Variants.thousandg2015aug_all,
Variants.ExAC_ALL,
FATHMM_pred.prediction,
LRT_pred.prediction,
MetaLR_pred.prediction,
MetaSVM_pred.prediction,
MutationAssessor_pred.prediction,
MutationTaster_pred.prediction,
PROVEAN_pred.prediction,
Polyphen2_HDIV_pred.prediction,
Polyphen2_HVAR_pred.prediction,
SIFT_pred.prediction,
fathmm_MKL_coding_pred.prediction,
sum(PredictionStats.PredictorValue)
```

For the predictions, return the actual text value. And the last column is the sum of all the mapped prediction scores for a given variant!



In [30]:
# def fetch_variant(conn, CHROM, POS, ID, REF, ALT):
#     query_sql = q14_query_variant_sql( CHROM, POS, ID, REF, ALT)
#     c = conn.cursor()
#     c.execute(query_sql)
#     conn.commit()
#     rows
    
# def q14_query_variant_sql(CHROM, POS, ID, REF, ALT):
    
#     query_column = '''Variants.CHROM ,Variants.POS,Variants.ID, Variants.REF, Variants.ALT,Variants.QUAL, Variants.FILTER, Variants.thousandg2015aug_all,Variants.ExAC_ALL,FATHMM_pred.prediction,LRT_pred.prediction, MetaLR_pred.prediction, MetaSVM_pred.prediction,MutationAssessor_pred.prediction, MutationTaster_pred.prediction,PROVEAN_pred.prediction, Polyphen2_HDIV_pred.prediction, Polyphen2_HVAR_pred.prediction,SIFT_pred.prediction, fathmm_MKL_coding_pred.prediction,sum(PredictionStats.PredictorValue) '''
#     query_table = '''Variants, FATHMM_pred , LRT_pred , MetaLR_pred, MetaSVM_pred ,  MutationAssessor_pred , MutationTaster_pred, PROVEAN_pred , Polyphen2_HDIV_pred , Polyphen2_HVAR_pred, SIFT_pred,  fathmm_MKL_coding_pred , PredictionStats '''
#     query_condition = " Variants.CHROM = " + str(CHROM) + " AND Variants.POS = " + str(POS) + " AND Variants.ID = '" + str(ID) + "' AND Variants.REF = '" + str(REF) + "' AND Variants.ALT = '" + str(ALT) + "' "
    
#     query_join_condition = q14_join_line_condition()
    
#     query_sql = "SELECT " + query_column + " FROM " + query_table + \
#             " WHERE " + query_condition  + query_join_condition
#     print(query_sql)
#     return query_sql 


# def q14_join_line_condition ():
#     query_join_condition_FATHMM_pred = " AND FATHMM_pred.FATHMM_predID = Variants.FATHMM_predID " 
#     query_join_condition_LRT_pred =  " AND LRT_pred.LRT_predID = Variants.LRT_predID " 
#     query_join_condition_MetaLR_pred=  " AND MetaLR_pred.MetaLR_predID = Variants.MetaLR_predID " 
#     query_join_condition_MetaSVM_pred=  " AND MetaSVM_pred.MetaSVM_predID = Variants.MetaSVM_predID " 
#     query_join_condition_MutationAssessor_pred=  " AND MutationAssessor_pred.MutationAssessor_predID = Variants.MutationAssessor_predID " 
#     query_join_condition_MutationTaster_pred=  " AND MutationTaster_pred.MutationTaster_predID = Variants.MutationTaster_predID " 
#     query_join_condition_PROVEAN_pred=  " AND PROVEAN_pred.PROVEAN_predID = Variants.PROVEAN_predID " 
#     query_join_condition_Polyphen2_HDIV_pred=  " AND Polyphen2_HDIV_pred.Polyphen2_HDIV_predID = Variants.Polyphen2_HDIV_predID " 
#     query_join_condition_Polyphen2_HVAR_pred=  " AND Polyphen2_HVAR_pred.Polyphen2_HVAR_predID = Variants.Polyphen2_HVAR_predID " 
#     query_join_condition_SIFT_pred =  " AND SIFT_pred.SIFT_predID = Variants.SIFT_predID " 
#     query_join_condition_fathmm_MKL_coding_pred  =  " AND fathmm_MKL_coding_pred.fathmm_MKL_coding_predID = Variants.fathmm_MKL_coding_predID " 
    
#     return_query1 = query_join_condition_FATHMM_pred + query_join_condition_LRT_pred + query_join_condition_MetaLR_pred + query_join_condition_MetaSVM_pred
#     return_query2 = query_join_condition_MutationAssessor_pred + query_join_condition_MutationTaster_pred + query_join_condition_PROVEAN_pred
#     return_query3 = query_join_condition_Polyphen2_HDIV_pred + query_join_condition_Polyphen2_HVAR_pred + query_join_condition_SIFT_pred + query_join_condition_fathmm_MKL_coding_pred
#     final_return = return_query1 + return_query2 + return_query3 
#     return final_return 


def fetch_variant(conn, CHROM, POS, ID, REF, ALT):
    """
    See Part 14 description
    """
    # a bunch of LEFT.JOIN name of predictor USING(predictorId)
   
    cur = conn.cursor()
   
    s = """SELECT
        Variants.CHROM,
        Variants.POS,
        Variants.ID,
        Variants.REF,
        Variants.ALT,
        Variants.QUAL,
        Variants.FILTER,
        Variants.thousandg2015aug_all,
        Variants.ExAC_ALL,
        FATHMM_pred.prediction,
        LRT_pred.prediction,
        MetaLR_pred.prediction,
        MetaSVM_pred.prediction,
        MutationAssessor_pred.prediction,
        MutationTaster_pred.prediction,
        PROVEAN_pred.prediction,
        Polyphen2_HDIV_pred.prediction,
        Polyphen2_HVAR_pred.prediction,
        SIFT_pred.prediction,
        fathmm_MKL_coding_pred.prediction,
        sum(predictionStats.PredictorValue)
       
        FROM Variants
        LEFT JOIN FATHMM_pred USING(FATHMM_predID)
        LEFT JOIN LRT_pred USING(LRT_predID)
        LEFT JOIN MetaLR_pred USING(MetaLR_predID)
        LEFT JOIN MetaSVM_pred USING(MetaSVM_predID)
        LEFT JOIN MutationAssessor_pred USING(MutationAssessor_predID)
        LEFT JOIN MutationTaster_pred USING(MutationTaster_predID)
        LEFT JOIN PROVEAN_pred USING(PROVEAN_predID)
        LEFT JOIN Polyphen2_HDIV_pred USING(Polyphen2_HDIV_predID)
        LEFT JOIN Polyphen2_HVAR_pred USING(Polyphen2_HVAR_predID)
        LEFT JOIN SIFT_pred USING(SIFT_predID)
        LEFT JOIN fathmm_MKL_coding_pred USING(fathmm_MKL_coding_predID)
        LEFT JOIN predictionStats USING(VariantID)
        WHERE CHROM = '%s' AND POS = %d AND ID = '%s' AND REF = '%s' AND ALT = '%s'
        """ %(CHROM, POS, ID, REF, ALT)
   

    cur.execute(s)
    conn.commit()
    return(cur.fetchall()[0])

In [31]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert fetch_variant(conn, '22', 25599849, 'rs17670506', 'G', 'A') == ('22', 25599849, 'rs17670506', 'G', 'A', 3124.91, 'PASS', 0.0251597, 0.0425, 'D', 'D', 'T', 'T', 'M', 'D', 'D', 'D', 'D', 'D', 'D', 8.5)
conn.close()

In [32]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert fetch_variant(conn, 'X', 2836184, 'rs73632976', 'C', 'T') == ('X', 2836184, 'rs73632976', 'C', 'T', 1892.12, 'PASS', None, 0.0427, 'D', 'U', 'D', 'T', 'M', 'P', 'D', 'P', 'P', 'D', 'D', 6.5)
conn.close()

In [33]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert fetch_variant(conn, '5', 155935708, 'rs45559835', 'G', 'A') == ('5', 155935708, 'rs45559835', 'G', 'A', 1577.12, 'PASS', 0.0189696, 0.0451, 'D', 'D', 'T', 'T', 'L', 'D', 'D', 'P', 'B', 'T', 'D', 5.75)
conn.close()

In [34]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert fetch_variant(conn, '4', 123416186, '.', 'A', 'G') == ('4', 123416186, '.', 'A', 'G', 23.25, 'PASS', None, None, None, None, None, None, None, None, None, None, None, None, None, None)
conn.close()

# Part 15 (10 Points)
Write a function that returns the variant with the highest predictor score sum. 

Return the variant info the following order:
```
Variants.CHROM,
Variants.POS,
Variants.ID,
Variants.REF,
Variants.ALT,
Variants.QUAL,
Variants.FILTER,
Variants.thousandg2015aug_all,
Variants.ExAC_ALL,
FATHMM_pred.prediction,
LRT_pred.prediction,
MetaLR_pred.prediction,
MetaSVM_pred.prediction,
MutationAssessor_pred.prediction,
MutationTaster_pred.prediction,
PROVEAN_pred.prediction,
Polyphen2_HDIV_pred.prediction,
Polyphen2_HVAR_pred.prediction,
SIFT_pred.prediction,
fathmm_MKL_coding_pred.prediction,
sum(PredictionStats.PredictorValue)
        
```

Again, return the predictor text values and the last column is the sum of the prediction values. 

In [36]:
# def variant_with_highest_sum_of_predictor_value(conn):


def variant_with_highest_sum_of_predictor_value(conn):
    """
    See part 15 description
    """
    cur = conn.cursor()
    cur.execute("""SELECT VariantID, sum(PredictorValue) FROM predictionStats GROUP BY VariantID""")
#     conn.commit()
   
    variant_id = cur.fetchone()[0]
   
    cur = conn.cursor()
   
    cur.execute("""SELECT
                Variants.CHROM,
                Variants.POS,
                Variants.ID,
                Variants.REF,
                Variants.ALT,
                Variants.QUAL,
                Variants.FILTER,
                Variants.thousandg2015aug_all,
                Variants.ExAC_ALL,
                FATHMM_pred.prediction,
                LRT_pred.prediction,
                MetaLR_pred.prediction,
                MetaSVM_pred.prediction,
                MutationAssessor_pred.prediction,
                MutationTaster_pred.prediction,
                PROVEAN_pred.prediction,
                Polyphen2_HDIV_pred.prediction,
                Polyphen2_HVAR_pred.prediction,
                SIFT_pred.prediction,
                fathmm_MKL_coding_pred.prediction,
                sum(predictionStats.PredictorValue)
               
                FROM Variants
                LEFT JOIN FATHMM_pred USING(FATHMM_predID)
                LEFT JOIN LRT_pred USING(LRT_predID)
                LEFT JOIN MetaLR_pred USING(MetaLR_predID)
                LEFT JOIN MetaSVM_pred USING(MetaSVM_predID)
                LEFT JOIN MutationAssessor_pred USING(MutationAssessor_predID)
                LEFT JOIN MutationTaster_pred USING(MutationTaster_predID)
                LEFT JOIN PROVEAN_pred USING(PROVEAN_predID)
                LEFT JOIN Polyphen2_HDIV_pred USING(Polyphen2_HDIV_predID)
                LEFT JOIN Polyphen2_HVAR_pred USING(Polyphen2_HVAR_predID)
                LEFT JOIN SIFT_pred USING(SIFT_predID)
                LEFT JOIN fathmm_MKL_coding_pred USING(fathmm_MKL_coding_predID)
                LEFT JOIN predictionStats USING(VariantID)
                WHERE VariantID = %s
                ;"""%(variant_id))
   
#     conn.commit()
    return cur.fetchone()

In [37]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert variant_with_highest_sum_of_predictor_value(conn) == ('7', 87837848, '.', 'C', 'A', 418.25, 'PASS', None, None, 'T', 'D', 'D', 'D', 'H', 'D', 'D', 'D', 'D', 'D', 'D', 10.0)
conn.close()